In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [2]:
# get or create Spark session

app_name = "amazon"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv("file://"+SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), header=True,sep='\t', inferSchema= True)

In [4]:
df.count()

Out[4]: 9002021

In [5]:
customers=df.groupBy(df.customer_id).count()
customers_df=customers.selectExpr('customer_id','count as customer_count')

In [6]:
review_id_table_df=df.selectExpr("review_id", "customer_id","product_id","product_parent","review_date")
products_df=df.selectExpr("product_id", "product_title")
vine_table_df=df.selectExpr("review_id", "star_rating","helpful_votes","total_votes","vine")


In [7]:
customers_df.head()

Out[7]: Row(customer_id=46909180, customer_count=6)

In [8]:
review_id_table_df.head()

Out[8]: Row(review_id='R3W4P9UBGNGH1U', customer_id=16414143, product_id='B00YL0EKWE', product_parent=852431543, review_date=datetime.datetime(2015, 8, 31, 0, 0))

In [9]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])

In [10]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}

In [11]:
table = "review_id_table"
mode = "append" # options are: error, append, overwrite
review_id_table_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [12]:
table = "products"
mode = "append" # options are: error, append, overwrite
products_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [13]:
table = "customers"
mode = "append" # options are: error, append, overwrite
customers_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [14]:
table = "vine_table"
mode = "append" # options are: error, append, overwrite
vine_table_df.write.jdbc(jdbcUrl, table, mode, connectionProperties)